In [1]:
import numpy as np
from sklearn.neighbors import KDTree

In [2]:
class Node:
    def __init__(self, nid: str, x: str, y: str, z:str):
        self.nid = int(nid)
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
        self.coordinates = np.array([self.x,self.y,self.z])
        
class CQUAD4:
    def __init__(self, eid: str, n1: str, n2: str, n3: str, n4: str):
        self.eid = int(eid)
        self.n1 = int(n1)
        self.n2 = int(n2)
        self.n3 = int(n3)
        self.n4 = int(n4)
        self.attached_nodes = np.array([self.n1,self.n2,self.n3,self.n4])
        
    def face_centroid(self, nodes):
        centroid = np.array([0.,0.,0.])
        
        for nid in self.attached_nodes:
            centroid += nodes[str(nid)].coordinates
               
        centroid = centroid/len(self.attached_nodes)
        
        return centroid

class CBUSH:
    def __init__(self, eid: str, GA: str, GB: str):
        self.eid = int(eid)
        self.GA = int(GA)
        self.GB = int(GB)
        self.attached_nodes = np.array([self.GA,self.GB])

In [3]:
infile_fast = 'FASTENER.dat'

infile_lst = ['TOP.dat', 'BOTTOM.dat']

In [4]:
with open(infile_fast) as f:
    lines = [line.strip() for line in f]
    
fast_nodes = {}
cbush = {}

for line in lines:
    if line.startswith('GRID'):
        nid = line[8:16].strip()
        x = line[24:32]
        y = line[32:40]
        z = line[40:48]
        fast_nodes[nid] = Node(nid,x.strip(),y.strip(),z.strip())
    elif line.startswith('CBUSH'):
        eid = line[8:16].strip()
        GA = line[24:32]
        GB = line[32:40]
        cbush[eid] = CBUSH(eid,GA.strip(),GB.strip())

In [5]:
#for eid in sorted(cbush.keys()):
#    print(eid, cbush[eid].attached_nodes)
    
#for nid in sorted(fast_nodes.keys()):
#    print(nid, fast_nodes[nid].coordinates)

# pnt = np.array([fast_nodes['3000'].coordinates])

In [28]:
tst_dict = {}

dist_tol = 1

for infile in infile_lst:
    
    with open(infile) as f:
        lines = [line.strip() for line in f]
        
    nodes = {}
    cquad4 = {}

    for line in lines:
        if line.startswith('GRID'):
            nid = line[8:16].strip()
            x = line[24:32]
            y = line[32:40]
            z = line[40:48]
            nodes[nid] = Node(nid,x.strip(),y.strip(),z.strip())
        elif line.startswith('CQUAD4'):
            eid = line[8:16].strip()
            n1 = line[24:32]
            n2 = line[32:40]
            n3 = line[40:48]
            n4 = line[48:56]
            cquad4[eid] = CQUAD4(eid,n1.strip(),n2.strip(),n3.strip(),n4.strip())


    centroids = np.zeros([len(nodes),3])

    for i,eid in enumerate(sorted(cquad4.keys())):
        centroids[i] = cquad4[eid].face_centroid(nodes)

    tree = KDTree(centroids,leaf_size=40,metric='euclidean')
        
    for nid in sorted(fast_nodes.keys()):
        
        pnt = np.array([fast_nodes[nid].coordinates])
        
        dist,ind = tree.query(pnt,k=1)

        if dist[0][0] < dist_tol:
            pierced_elem = []
            for i in ind[0]:
                for eid in sorted(cquad4.keys()):
                    if (cquad4[eid].face_centroid(nodes) == centroids[i]).all():
                        pierced_elem.append(eid)

            print(infile,
                  nid,
                  'Pierced element id:',pierced_elem,
                  'Attached nodes:',cquad4[pierced_elem[0]].attached_nodes)
            
            if not infile in tst_dict:
                tst_dict[infile] = [nid, pierced_elem[0], cquad4[pierced_elem[0]].attached_nodes]
            else:
                tst_dict[infile].extend(nid, pierced_elem[0], cquad4[pierced_elem[0]].attached_nodes)
        

TOP.dat 3001 Pierced element id: ['2007'] Attached nodes: [2042 2038 2037 2041]
BOTTOM.dat 3000 Pierced element id: ['1017'] Attached nodes: [1034 1030 1029 1033]


In [29]:
print(tst_dict)

{'TOP.dat': ['3001', '2007', array([2042, 2038, 2037, 2041])], 'BOTTOM.dat': ['3000', '1017', array([1034, 1030, 1029, 1033])]}


In [ ]:
tst_dict = {}

tst_dict['eid'] = ['nid','Part']

In [ ]:
#if not nid in load:
#                load[nid] = [ Load(nid, lineSplit[1], lineSplit[2]) ] 
#            else:
#load[nid].append(Load(nid, lineSplit[1], lineSplit[2]))

In [ ]:
print(tst_dict)

In [ ]:
tst_dict['eid'].extend(['nid2','Part2'])

In [ ]:
print(tst_dict)